In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import os

# List of files to load (ensure these files are in the same directory as the notebook)
files = [
    'AWS FinOps 2025.xlsx - US Region.csv',
    'AWS FinOps 2025.xlsx - APAC Region.csv',
    'AWS FinOps 2025.xlsx - EU Region.csv'
]

all_data = []

# --- 1. Load, Combine, and Prepare Data ---
for file in files:
    try:
        # Extract region name from the filename
        region_name = file.split(' - ')[1].replace(' Region.csv', '')
        df = pd.read_csv(file)
        df['Region'] = region_name
        all_data.append(df)
    except FileNotFoundError:
        print(f"File not found: {file}. Skipping.")

if not all_data:
    print("No data files loaded. Exiting.")
    # Exit or raise an error if no data is loaded, depending on environment

df_combined = pd.concat(all_data, ignore_index=True)

# Define month order for melting (assuming the columns are Jan through Oct)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct']

# Melt the DataFrame from wide (months as columns) to long format (single month column)
df_long = df_combined.melt(
    id_vars=['Service', 'Region'],
    value_vars=months,
    var_name='Month',
    value_name='Cost'
)

# Convert 'Cost' to numeric
df_long['Cost'] = pd.to_numeric(df_long['Cost'])

# --- 2. Feature Engineering ---

# Map month names to numerical values for the model (Jan=1, Feb=2, etc.)
month_map = {month: i + 1 for i, month in enumerate(months)}
df_long['Month_Num'] = df_long['Month'].map(month_map)

# One-hot encode categorical features ('Service' and 'Region')
df_encoded = pd.get_dummies(df_long, columns=['Service', 'Region'], drop_first=True)

# Display the first few rows of the prepared data
print("--- Prepared Data Head ---")
print(df_encoded.head())
print("\n")


# --- 3. Model Training Data Prep ---

# Define features (X) and target (y)
# X contains the numerical month and the one-hot encoded service/region columns
X = df_encoded.drop(columns=['Month', 'Cost'])
y = df_encoded['Cost']

# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 4. Model Training and Prediction (Random Forest Regressor) ---

# Initialize and train the Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
print("Training the Random Forest Regressor model...")
model.fit(X_train, y_train)
print("Training complete.")

# Make predictions on the test set
y_pred = model.predict(X_test)

# --- 5. Evaluation ---

# Calculate R-squared score and Mean Absolute Error
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\n--- Model Performance ---")
print(f"R-squared ($R^2$): {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Verify the target accuracy range (70-80% R2, or 0.7 to 0.8)
if 0.7 <= r2 <= 0.8:
    print("Accuracy is within the requested 70% to 80% range (based on R-squared).")
elif r2 > 0.8:
    print("Accuracy (R-squared) is excellent, exceeding the requested range.")
else:
    print("Accuracy (R-squared) is below the requested 70% range. Consider more complex features or models.")


# --- 6. Sample Predictions for Testing ---
# Create a DataFrame to view actual vs. predicted values
comparison_df = pd.DataFrame({'Actual Cost': y_test, 'Predicted Cost': y_pred})
comparison_df['Absolute Error'] = abs(comparison_df['Actual Cost'] - comparison_df['Predicted Cost'])
comparison_df = comparison_df.sort_values(by='Absolute Error', ascending=False)

print("\n--- Sample of Predictions (Top 5 Worst Errors) ---")
print(comparison_df.head())